<a href="https://colab.research.google.com/github/BenxiaHu/DeepLearning/blob/main/pytorch_1D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network


Dependencies:
* torch: 0.1.11
* matplotlib

# step0: import python packages

In [14]:
import torch
import torch.nn as nn
import torch.utils.data as Data
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch.nn.functional as Fun
from numpy import array
import pandas as pd
import numpy as np
from numpy import argmax
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import requests
from torch.nn import LogSoftmax
torch.manual_seed(1)



# step1: import data

In [15]:
seqid="https://raw.githubusercontent.com/BenxiaHu/DeepLearning/main/K562_MPRA_Silencers.txt"
print(seqid)
seqid = pd.read_csv(seqid,header=0,sep="\t")
seqid["peakid"] = seqid['chr'] +":"+ seqid["start"].astype(str) +"-"+ seqid["end"].astype(str)
print(seqid)
seqid = seqid[["peakid","silencer"]]
print(seqid)

fastaid="https://raw.githubusercontent.com/BenxiaHu/DeepLearning/main/K562_MPRA_Silencers.fasta"
fastaid = pd.read_table(fastaid,header=None,sep="\t")
print(fastaid)
fastaid.rename(columns={0:"peakid",1:"fasta"},inplace=True)

result = pd.merge(fastaid, seqid, how='inner', on=['peakid', 'peakid'])

result[["fasta"]] = result[["fasta"]].apply(lambda x: x.astype(str).str.upper())

result = result[["fasta","silencer"]]

#label="https://raw.githubusercontent.com/BenxiaHu/DeepLearning/main/labels.txt"
#label = pd.read_table(labelid,header=None)
input = result[["fasta"]]
print(input.shape)
label = result[["silencer"]]

# Convert input to one-hot code
#def one_hot_encoding(seq):
#    mapping = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'T': [0, 0, 0, 1]}
#    one_hot = [mapping[nuc] for nuc in seq]
#    return np.array(one_hot)

#one_hot_input = np.array([one_hot_encoding(seq) for seq in input[0]])
#
#one_hot_input = torch.from_numpy(one_hot_input)
#one_hot_input = one_hot_input.permute(0, 2, 1)
#print(one_hot_input.shape)

# Split the data into training, test, and prediction sets
# Split the data into training, testing, and prediction sets
#x_train, x_test, y_train, y_test = train_test_split(one_hot_input, label, test_size=0.3, random_state=42)
#x_test, x_pred, y_test, y_pred = train_test_split(x_test, y_test, test_size=0.67, random_state=42)
#print(x_train.shape)
#print(x_test.shape)


DNA = np.zeros(shape=(len(input),len(input["fasta"][0]),4))
labelid = np.zeros(shape=(len(input),))
#print(DNA.shape)
#print(labelid.shape)

for i in range(input.shape[0]):
    seq_array = array(list(input["fasta"][i]))
    #integer encode the sequence
    label_encoder = LabelEncoder()
    integer_encoded_seq = label_encoder.fit_transform(seq_array)
    #one hot the sequence
    onehot_encoder = OneHotEncoder(sparse_output=False)
    #reshape because that's what OneHotEncoder likes
    #print(integer_encoded_seq.shape)
    integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
    #print(integer_encoded_seq.shape)
    onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)
    #print(onehot_encoded_seq.shape)
    #print(len(onehot_encoded_seq))
    DNA[i] = onehot_encoded_seq
    #DNA[i] = onehot_encoded_seq.reshape(4,len(onehot_encoded_seq))
    labelid[i] = label["silencer"][i]

#print(DNA.shape)
#print(labelid.shape)

DNA = torch.tensor(DNA)
DNA = DNA.permute(0, 2, 1)
labelid =  torch.tensor(labelid)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    DNA = DNA.to("cuda:0")
    labelid =  labelid.to("cuda:0")
#print(DNA.is_cuda)

#print(np.shape(DNA))
#embed_x = embed_x.permute(0, 2, 1)

https://raw.githubusercontent.com/BenxiaHu/DeepLearning/main/K562_MPRA_Silencers.txt
        chr      start        end  silencer                     peakid
0      chr1  113408175  113408375         1   chr1:113408175-113408375
1     chr10    6107715    6107915         1      chr10:6107715-6107915
2     chr10   15538675   15538875         1    chr10:15538675-15538875
3     chr11  114339756  114339956         1  chr11:114339756-114339956
4     chr17   41121376   41121576         1    chr17:41121376-41121576
...     ...        ...        ...       ...                        ...
7227   chrX   93929096   93929296         0     chrX:93929096-93929296
7228   chrX  101946395  101946595         0   chrX:101946395-101946595
7229   chrX  130859105  130859305         0   chrX:130859105-130859305
7230   chrX  131424075  131424275         0   chrX:131424075-131424275
7231   chrX  141256436  141256636         0   chrX:141256436-141256636

[7232 rows x 5 columns]
                         peakid  silen

# step2: split the data into training, test and prediction data

In [16]:
#input_tensor = DNA
#label_tensor = labelid
# pick 1400 samples as training data
#print(input_tensor[0:1400,:,:].shape)
#print(label_tensor[0:1400].shape)

# Split the data into training, test, and prediction sets
x_train, x_test, y_train, y_test = train_test_split(DNA, labelid, test_size=0.3, random_state=42)

#print(x_train.shape)
# Hyper Parameters
EPOCH = 10              # train the training data n times, to save time, we just train 1 epoch
BATCH_SIZE = 10
LR = 0.1                 # learning rate
torch_dataset = Data.TensorDataset(x_train, y_train)
train_loader = Data.DataLoader(dataset=torch_dataset, batch_size=BATCH_SIZE, shuffle=True)
torch_dataset2 = Data.TensorDataset(x_test, y_test)
test_loader = Data.DataLoader(dataset=torch_dataset2, shuffle=True)
              
# Data Loader for easy mini-batch return in training, the image batch shape will be (100, 1, 50, 4)

#for i, j in enumerate(train_loader):
    #x, y = j
    #print('batch:{0} x:{1}  y: {2}'.format(i, x, y))
    #print(i)
    #print(x.shape)
    #print(y.shape)

# pick 400 samples as prediction data
#test_x2 = Variable(input_tensor[1600:2000,:,:]).type(torch.FloatTensor)
#test_y2 = label_tensor[1600:2000]


# step4: build the 1D-CNN model

In [19]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # input shape (50,4,200)
            nn.Conv1d(
                in_channels=4,              # input height
                out_channels=1,            # n_filters
                kernel_size=21,             # filter size
                stride=1,                   # filter movement/step
                padding=0                   # if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (32, 1, 190)
            nn.ReLU(),                      # activation
            nn.MaxPool1d(
                kernel_size=3,
                stride=1,
                padding=0)                  # choose max value in 1x2 area, output shape (32, 1, 178)
        )
        self.conv2 = nn.Sequential(         # input shape (32, 1, 178)
            nn.Conv1d(
                in_channels=1,             # input height
                out_channels=1,            # n_filters
                kernel_size=21,             # filter size
                stride=1,                   # filter movement/step
                padding=0                   # output shape (32, 1, 158)
            ),
            nn.ReLU(),                      # activation
            nn.MaxPool1d(
                kernel_size=3,
                stride=1,
                padding=0)                # output shape (32, 1, 156)
        )
        self.conv3 = nn.Sequential(       # input shape (32, 1, 156)
            nn.Conv1d(
                in_channels=1,           # input height
                out_channels=1,          # n_filters
                kernel_size=21,           # filter size
                stride=1,                 # filter movement/step
                padding=0                 # output shape (16, 1, 136)
            ),
            nn.ReLU(),                    # activation
            nn.MaxPool1d(
                kernel_size=3,
                stride=1,
                padding=0)                # output shape (16, 1, 134)
        )
        self.conv4 = nn.Sequential(       # input shape (16, 1, 134)
            nn.Conv1d(
                in_channels=1,            # input height
                out_channels=1,           # n_filters
                kernel_size=21,           # filter size
                stride=1,                 # filter movement/step
                padding=0                 # output shape (16, 1, 114)
            ),
            nn.ReLU(),                    # activation
            nn.MaxPool1d(
                kernel_size=3,
                stride=1,
                padding=0)                # output shape (16, 1, 112)
        )
        self.conv5 = nn.Sequential(       # input shape (16, 1, 112)
            nn.Conv1d(
                in_channels=1,            # input height
                out_channels=1,           # n_filters
                kernel_size=21,           # filter size
                stride=1,                 # filter movement/step
                padding=0                 # output shape (16, 1, 92)
            ),
            nn.ReLU(),                    # activation
            nn.MaxPool1d(
                kernel_size=3,
                stride=1,
                padding=0)                # output shape (16, 1, 90)
        )
        self.conv6 = nn.Sequential(       # input shape (16, 1, 90)
            nn.Conv1d(
                in_channels=1,            # input height
                out_channels=1,           # n_filters
                kernel_size=21,           # filter size
                stride=1,                 # filter movement/step
                padding=0                 # output shape (16, 1, 70)
            ),
            nn.ReLU(),                    # activation
            nn.MaxPool1d(
                kernel_size=3,
                stride=1,
                padding=0)                # output shape (8, 1, 68)
        )
        self.conv7 = nn.Sequential(       # input shape (8, 1, 68)
            nn.Conv1d(
                in_channels=1,            # input height
                out_channels=1,           # n_filters
                kernel_size=21,           # filter size
                stride=1,                 # filter movement/step
                padding=0                 # output shape (8, 1, 48)
            ),
            nn.ReLU(),                    # activation
            nn.MaxPool1d(
                kernel_size=3,
                stride=1,
                padding=0)                # output shape (8, 1, 46)
        )
        self.conv8 = nn.Sequential(       # input shape (8, 1, 46)
            nn.Conv1d(
                in_channels=1,            # input height
                out_channels=1,           # n_filters
                kernel_size=21,           # filter size
                stride=1,                 # filter movement/step
                padding=0                 # output shape (8, 1, 26)
            ),
            nn.ReLU(),                    # activation
            nn.MaxPool1d(
                kernel_size=3,
                stride=1,
                padding=0)                # output shape (8, 1, 24)
        )
        self.conv9 = nn.Sequential(       # input shape (8, 1, 24)
            nn.Conv1d(
                in_channels=1,            # input height
                out_channels=1,           # n_filters
                kernel_size=21,           # filter size
                stride=1,                 # filter movement/step
                padding=0                 # output shape (8, 1, 30)
            ),
            nn.ReLU(),                    # activation
            nn.MaxPool1d(
                kernel_size=3,
                stride=1,
                padding=0)                # output shape (4, 1, 20)
        )
        #self.relu = nn.ReLU()
        self.fc1 = nn.Linear(1*24, 10)     # fully connected layer
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(10, 2)       # fully connected layer
        self.softmax = nn.Softmax(dim=0)

    def forward(self, x):
        x = self.conv1(x)
        #print(x.size())            # torch.Size([50, 32, 38])
        x = self.conv2(x)
        #print(x.size())            # torch.Size([50, 16, 26])
        x = self.conv3(x)
        #print(x.size())            # torch.Size([50, 8, 14])
        x = self.conv4(x)
        #print(x.size())            # torch.Size([50, 4, 2])
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        #print(x.size())
        #x = self.conv9(x)
        x = torch.flatten(x,1)  # flatten the output to 2 dimensions from 3 dimensions
        x = self.fc1(x)
        #print(x.size())            # torch.Size([50, 10])
        x = self.relu(x)
        #print(x.size())            # torch.Size([50, 10])
        x = self.dropout(x) 
        #x = x.view(x.size(0), -1)   # flatten the output 1 dimension from 2 dimensions
        x = self.fc2(x)
        #print(x)
        x = torch.flatten(x,1)
        out = self.softmax(x)
        return out              # return x for visualization


cnn = CNN()
#print(cnn)  # net architecture

optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

# running the CNN model

In [31]:
# training and testing
optimizer.zero_grad()           # clear gradients for this training step
train_losses = []
trainCorrect = 0
for epoch in range(EPOCH):
    running_loss = 0.0
    for i, (b_x, b_y) in enumerate(train_loader):  # gives batch data, normalize x when iterate train_loader
        #print(b_x.shape)
        b_x = Variable(b_x).type(torch.FloatTensor)
        b_y = b_y.type(torch.FloatTensor)
        optimizer.zero_grad()           # clear gradients for this training step
        output = cnn(b_x)               # cnn output
        preds, predsid = torch.max(output,1) 
        loss = loss_func(preds, b_y)   # cross entropy loss
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients
        # print statistics
        running_loss += loss.item()
        train_losses.append(loss.item())
        trainCorrect += (predsid == b_y).type(torch.float).sum().item()

        if i % 10 == 0:    # print every 100 mini-batches
            #print(i)
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss}')
            running_loss = 0.0

print('Finished Training')

#trainSteps = len(train_loader.dataset) // BATCH_SIZE
# calculate the average training and validation loss
#avgTrainLoss = sum(train_losses) / trainSteps
# calculate the training and validation accuracy
#trainCorrect = trainCorrect / trainSteps

# Let’s quickly save our trained model:
# Plot the training and test losses over time
plt.plot(train_losses, label='Train Loss')
plt.legend()
plt.xlabel("# of epoch")
plt.ylabel("loss")
plt.show()
!pwd
torch.save(cnn.state_dict(), "./content")



[1,     1] loss: 4.605170249938965
[1,    11] loss: 89.80082273483276
[1,    21] loss: 105.91892004013062
[1,    31] loss: 94.40599250793457
[1,    41] loss: 115.12926149368286
[1,    51] loss: 94.40599489212036
[1,    61] loss: 101.31374883651733
[1,    71] loss: 94.40599203109741
[1,    81] loss: 87.49823665618896
[1,    91] loss: 101.31374883651733
[1,   101] loss: 89.80082368850708
[1,   111] loss: 85.19565153121948
[1,   121] loss: 94.40599298477173
[1,   131] loss: 80.59048175811768
[1,   141] loss: 96.70857858657837
[1,   151] loss: 103.61633539199829
[1,   161] loss: 96.70857954025269
[1,   171] loss: 96.70857810974121
[1,   181] loss: 75.98531103134155
[1,   191] loss: 99.01116514205933
[1,   201] loss: 108.22150564193726
[1,   211] loss: 105.91892051696777
[1,   221] loss: 71.38014030456543
[1,   231] loss: 85.19565153121948
[1,   241] loss: 92.1034083366394
[1,   251] loss: 99.01116466522217
[1,   261] loss: 110.52409029006958
[1,   271] loss: 94.40599346160889
[1,   281] lo

KeyboardInterrupt: ignored

# apply the model to the test data

In [ ]:
net = CNN()
net.load_state_dict(torch.load("./content"))
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for i, (b_x, b_y) in enumerate(test_loader):
        b_x = Variable(b_x).type(torch.FloatTensor)
        #print(b_x.shape)
        #b_y = b_y.type(torch.LongTensor)
        # calculate outputs by running the network
        outputs = net(b_x)
        #print(outputs[0].shape)
        #print(outputs[1].shape)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs[0], 1)
        total += b_y.size(0)
        correct += (predicted == b_y).sum().item()           

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
